# Crack identification procedure

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, \
    DICAlignedGrid, DICGrid
from bmcs_shear.dic_crack.dic_crack_list2 import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

In [ ]:
from bmcs_shear.dic_crack import sz_tests_series_2023

In [ ]:
test = sz_tests_series_2023.B10_TV1
dic_points = DICInpUnstructuredPoints(**test)
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, **test)

In [ ]:
dic_grid.interact()

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, **test)
dsf.tmodel_.trait_set(**test)
dsf.tmodel_.omega_fn_.trait_set(kappa_0=0.002, kappa_f=0.0028);
dcl = DICCrackList(dsf=dsf, **test)

To reproduce the history of cracking, the detection is running in several 
steps:
 - To distinguish the primary cracks, the crack detection uses the damage state
   at the specified fraction of load expressed by the ratio $t_\mathrm{prim}$.
 - Cracks that start at neighbouring positions and merge along the path
   are identified by imposing a criterion of a minimum distance between
   the crack tips
 - The tip range of the primary cracks is then detected in the loading range
   $t \in [t_\mathrm{prim}, 1]$
 - Finally, the path of secondary cracks is identified by scanning the damage
   along the primary cracks at the $t=1$.

In [ ]:
dcl.interact()

In [ ]:
X_CKa = dcl.X_CKa

In [ ]:
dcl.cracks[0].X_crc_1_Na

In [ ]:
%%capture

from matplotlib import animation, rc
from IPython.display import HTML
from traitlets import HasTraits
import matplotlib.gridspec as gridspec


fig = plt.figure(figsize=(8,3))

In [ ]:
class CrackListAnimator(HasTraits):
    
    
    def init(self):
        print('init')
        n_T = dsf.dic_grid.n_T
        self.t_dic_T = np.hstack([
            np.linspace(0,1,n_T),
            np.ones((int(0.5*n_T),))
        ])
    
    def subplots(self, fig):
        gs = gridspec.GridSpec(ncols=2, nrows=1,
                               width_ratios=[3, 1],
                               #wspace=0.5,
                               hspace=0.5,
                               # height_ratios=[2, 1]
                               )
        ax_dcl = fig.add_subplot(gs[0, 0])
        ax_FU = fig.add_subplot(gs[0, 1])
#        return fig.subplots(1,1)
#        return ax_dsf#, ax_FU
        return ax_dcl, ax_FU

    def plot(self, i):
        fig.clear()
        t = self.t_dic_T[i]
        print('t', t)
        axes = self.subplots(fig)
        dcl.dsf.dic_grid.t = t
        
        ax_dcl, ax_FU = axes
#        ax_dsf = axes
#        ax_FU = axes
        
        dcl.bd.plot_sz_bd(ax_dcl)
        dcl.dsf.plot_crack_detection_field(ax_dcl, fig)
        dcl.plot_primary_cracks(ax_dcl)
        dcl.critical_crack.plot_X_crc_t_Ka(ax_dcl, line_width=2, line_color='red', tip_color='red')
        for crack in dcl.cracks:
            crack.cor.trait_set(cor_marker_size=8, cor_marker_color='brown')
            crack.cor.plot_X_cor_t(ax_dcl)
        ax_dcl.axis('equal')
        ax_dcl.axis('off');
        dcl.dsf.dic_grid.plot_load_deflection(ax_FU)
        
    def mp4_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cracking_animation.gif")
    
    def html5_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [ ]:
# anim = CrackListAnimator()
# anim.init()
# anim.mp4_video()

In [ ]:
anim = CrackListAnimator()
anim.init()
with open('cracking_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

In [ ]:
html_video_file = open('cracking_animation.html','r')
HTML(html_video_file.read())

Given the step length $\Delta s$ get the damage values in the distance $s$ along an angle $\alpha_\min, \alpha_\max$

In [ ]:
dcl.cracks[3].cor.interact()

In [ ]:
test = sz_tests_series_2023.B6_TV1
dic_points = DICInpUnstructuredPoints(**test)
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, **test)

In [ ]:
dic_grid.data_dir

In [ ]:
import os.path as op
import pandas as pd
fos_data_csv = op.join(dic_grid.data_dir,'fos_data','B6_1_FOS_N.CSV')

In [ ]:
fos_TM = pd.read_table(fos_data_csv, sep=';', decimal=',')

In [ ]:
fos_TM_ = fos_TM.to_numpy()

In [ ]:
fos_TM_[0]

In [ ]:
fos_TM

In [ ]:
np.where(fos_TM_[:,1] == np.NaN)

In [ ]:
np.einsum('Eima,m->Eia', sig_Eima, w
          _m)